In [1]:
import nltk

In [2]:
import tensorflow as tf
from threading import Thread
from time import perf_counter
from transformers import GPT2LMHeadModel, GPT2Tokenizer

import os
import csv
import random

SEED = 42
ITERS = 2000
MIN_TOKENS = 45
MAX_TOKENS = 60
MODEL_NAME = "gpt2-medium"
DATASET_PATH = f"bias_llm/data/{MODEL_NAME}"
TOPICS = [
    "Indian climate",
    "Indian defense",
    "Indian economy",
    "Indian infrastructure",
]


def get_transformer() -> tuple:
    tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
    model = GPT2LMHeadModel.from_pretrained(
        MODEL_NAME, pad_token_id=tokenizer.eos_token_id,
    )
    print(model.num_parameters)
#     model.to('cuda')
    return tokenizer, model


def get_encoding(tokenizer: GPT2Tokenizer, prompt: str) -> dict:
    return tokenizer.encode(prompt, return_tensors="pt")


def get_text(tokenizer: GPT2Tokenizer, embedding: dict) -> str:
    return tokenizer.batch_decode(
        embedding, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]


def generate_text(tokenizer: GPT2Tokenizer, model: GPT2LMHeadModel, prompt) -> None:
    create_directory(prompt)
    create_csv_file(prompt)

    input_embedding = get_encoding(tokenizer, prompt)

    for i in range(ITERS):
        MAX_LEN = random.randint(MIN_TOKENS, MAX_TOKENS)
        print(f'Generating text for "{prompt}"... ({i + 1}/{ITERS})')

        output_embedding = model.generate(
            input_embedding,
            no_repeat_ngram_size=2,
            do_sample=True,
            max_length=MAX_LEN,
            temperature=0.9,
            top_k=40,
            top_p=0.95,
        )

        generated_text = get_text(tokenizer, output_embedding)
        append_to_csv_file(prompt, generated_text, i, MAX_LEN)


def create_directory(prompt: str) -> None:
    if not os.path.exists(f"{DATASET_PATH}/{prompt}"):
        os.makedirs(f"{DATASET_PATH}/{prompt}")

    print(f'Created directory for "{prompt}" at {DATASET_PATH}/{prompt}...')


def create_csv_file(prompt: str) -> None:
    fields = ["Prompt", "Generated Text", "i", "MAX_LEN"]

    with open(f"{DATASET_PATH}/{prompt}/data.csv", "w") as f:
        writer = csv.DictWriter(f, fieldnames=fields, lineterminator="\n")
        writer.writeheader()

    print(f'Created CSV file for "{prompt}" at {DATASET_PATH}/{prompt}/data.csv...')


def append_to_csv_file(prompt: str, generated_text: str, i: int, MAX_LEN: int) -> None:
    with open(f"{DATASET_PATH}/{prompt}/data.csv", "a") as f:
        writer = csv.writer(f)
        writer.writerow([prompt, generated_text, i, MAX_LEN])
        f.close()


def main() -> None:
    threads = []
    tokenizer, model = get_transformer()

    for topic in TOPICS:
        thread = Thread(
            target=generate_text,
            args=(
                tokenizer,
                model,
                topic,
            ),
        )
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

2023-09-16 07:14:15.949214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 07:14:16.089305: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-16 07:14:16.596057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-16 07:14:16.596117: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [3]:
import torch
torch.manual_seed(0)

start_time = perf_counter()
print("Starting...")
main()
print("Done!")
end_time = perf_counter()

print(f"Time Taken: {end_time - start_time:0.4f} second(s) elapsed")


Starting...
<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_drop

Generating text for "Indian defense"... (2/2000)
Generating text for "Indian climate"... (2/2000)
Generating text for "Indian infrastructure"... (2/2000)
Generating text for "Indian economy"... (2/2000)
Generating text for "Indian defense"... (3/2000)
Generating text for "Indian economy"... (3/2000)
Generating text for "Indian infrastructure"... (3/2000)
Generating text for "Indian climate"... (3/2000)
Generating text for "Indian infrastructure"... (4/2000)
Generating text for "Indian economy"... (4/2000)
Generating text for "Indian defense"... (4/2000)
Generating text for "Indian climate"... (4/2000)
Generating text for "Indian defense"... (5/2000)
Generating text for "Indian infrastructure"... (5/2000)
Generating text for "Indian economy"... (5/2000)
Generating text for "Indian climate"... (5/2000)
Generating text for "Indian defense"... (6/2000)
Generating text for "Indian economy"... (6/2000)
Generating text for "Indian infrastructure"... (6/2000)
Generating text for "Indian climat

Generating text for "Indian economy"... (42/2000)
Generating text for "Indian climate"... (40/2000)
Generating text for "Indian defense"... (41/2000)
Generating text for "Indian infrastructure"... (44/2000)
Generating text for "Indian economy"... (43/2000)
Generating text for "Indian climate"... (41/2000)
Generating text for "Indian defense"... (42/2000)
Generating text for "Indian infrastructure"... (45/2000)
Generating text for "Indian economy"... (44/2000)
Generating text for "Indian climate"... (42/2000)
Generating text for "Indian infrastructure"... (46/2000)
Generating text for "Indian defense"... (43/2000)
Generating text for "Indian economy"... (45/2000)
Generating text for "Indian climate"... (43/2000)
Generating text for "Indian infrastructure"... (47/2000)
Generating text for "Indian defense"... (44/2000)
Generating text for "Indian economy"... (46/2000)
Generating text for "Indian climate"... (44/2000)
Generating text for "Indian infrastructure"... (48/2000)
Generating text

Generating text for "Indian climate"... (78/2000)
Generating text for "Indian economy"... (82/2000)
Generating text for "Indian infrastructure"... (85/2000)
Generating text for "Indian defense"... (81/2000)
Generating text for "Indian climate"... (79/2000)
Generating text for "Indian infrastructure"... (86/2000)
Generating text for "Indian economy"... (83/2000)
Generating text for "Indian defense"... (82/2000)
Generating text for "Indian climate"... (80/2000)
Generating text for "Indian infrastructure"... (87/2000)
Generating text for "Indian economy"... (84/2000)
Generating text for "Indian climate"... (81/2000)
Generating text for "Indian defense"... (83/2000)
Generating text for "Indian infrastructure"... (88/2000)
Generating text for "Indian economy"... (85/2000)
Generating text for "Indian climate"... (82/2000)
Generating text for "Indian infrastructure"... (89/2000)
Generating text for "Indian defense"... (84/2000)
Generating text for "Indian economy"... (86/2000)
Generating text

Generating text for "Indian economy"... (121/2000)
Generating text for "Indian infrastructure"... (125/2000)
Generating text for "Indian infrastructure"... (126/2000)
Generating text for "Indian climate"... (118/2000)
Generating text for "Indian defense"... (119/2000)
Generating text for "Indian economy"... (122/2000)
Generating text for "Indian infrastructure"... (127/2000)
Generating text for "Indian climate"... (119/2000)
Generating text for "Indian defense"... (120/2000)
Generating text for "Indian economy"... (123/2000)
Generating text for "Indian infrastructure"... (128/2000)
Generating text for "Indian defense"... (121/2000)
Generating text for "Indian climate"... (120/2000)
Generating text for "Indian economy"... (124/2000)
Generating text for "Indian infrastructure"... (129/2000)
Generating text for "Indian defense"... (122/2000)
Generating text for "Indian climate"... (121/2000)
Generating text for "Indian infrastructure"... (130/2000)
Generating text for "Indian economy"... 

...

Generating text for "Indian economy"... (1834/2000)
Generating text for "Indian defense"... (1793/2000)
Generating text for "Indian infrastructure"... (1852/2000)
Generating text for "Indian climate"... (1804/2000)
Generating text for "Indian economy"... (1835/2000)
Generating text for "Indian defense"... (1794/2000)
Generating text for "Indian infrastructure"... (1853/2000)
Generating text for "Indian infrastructure"... (1854/2000)
Generating text for "Indian climate"... (1805/2000)
Generating text for "Indian economy"... (1836/2000)
Generating text for "Indian defense"... (1795/2000)
Generating text for "Indian infrastructure"... (1855/2000)
Generating text for "Indian economy"... (1837/2000)
Generating text for "Indian climate"... (1806/2000)
Generating text for "Indian defense"... (1796/2000)
Generating text for "Indian infrastructure"... (1856/2000)
Generating text for "Indian climate"... (1807/2000)
Generating text for "Indian economy"... (1838/2000)
Generating text for "Indian d

Generating text for "Indian economy"... (1873/2000)
Generating text for "Indian infrastructure"... (1891/2000)
Generating text for "Indian defense"... (1831/2000)
Generating text for "Indian climate"... (1841/2000)
Generating text for "Indian economy"... (1874/2000)
Generating text for "Indian climate"... (1842/2000)
Generating text for "Indian infrastructure"... (1892/2000)
Generating text for "Indian defense"... (1832/2000)
Generating text for "Indian infrastructure"... (1893/2000)
Generating text for "Indian economy"... (1875/2000)
Generating text for "Indian infrastructure"... (1894/2000)
Generating text for "Indian climate"... (1843/2000)
Generating text for "Indian defense"... (1833/2000)
Generating text for "Indian economy"... (1876/2000)
Generating text for "Indian climate"... (1844/2000)
Generating text for "Indian infrastructure"... (1895/2000)
Generating text for "Indian defense"... (1834/2000)
Generating text for "Indian economy"... (1877/2000)
Generating text for "Indian d

Generating text for "Indian infrastructure"... (1929/2000)
Generating text for "Indian defense"... (1868/2000)
Generating text for "Indian climate"... (1879/2000)
Generating text for "Indian economy"... (1913/2000)
Generating text for "Indian defense"... (1869/2000)
Generating text for "Indian infrastructure"... (1930/2000)
Generating text for "Indian climate"... (1880/2000)
Generating text for "Indian defense"... (1870/2000)
Generating text for "Indian economy"... (1914/2000)
Generating text for "Indian infrastructure"... (1931/2000)
Generating text for "Indian climate"... (1881/2000)
Generating text for "Indian defense"... (1871/2000)
Generating text for "Indian infrastructure"... (1932/2000)
Generating text for "Indian economy"... (1915/2000)
Generating text for "Indian climate"... (1882/2000)
Generating text for "Indian defense"... (1872/2000)
Generating text for "Indian infrastructure"... (1933/2000)
Generating text for "Indian climate"... (1883/2000)
Generating text for "Indian d

Generating text for "Indian infrastructure"... (1967/2000)
Generating text for "Indian climate"... (1916/2000)
Generating text for "Indian defense"... (1907/2000)
Generating text for "Indian infrastructure"... (1968/2000)
Generating text for "Indian economy"... (1952/2000)
Generating text for "Indian climate"... (1917/2000)
Generating text for "Indian defense"... (1908/2000)
Generating text for "Indian infrastructure"... (1969/2000)
Generating text for "Indian economy"... (1953/2000)
Generating text for "Indian climate"... (1918/2000)
Generating text for "Indian defense"... (1909/2000)
Generating text for "Indian infrastructure"... (1970/2000)
Generating text for "Indian economy"... (1954/2000)
Generating text for "Indian climate"... (1919/2000)
Generating text for "Indian defense"... (1910/2000)
Generating text for "Indian infrastructure"... (1971/2000)
Generating text for "Indian economy"... (1955/2000)
Generating text for "Indian climate"... (1920/2000)
Generating text for "Indian d

Generating text for "Indian defense"... (1947/2000)
Generating text for "Indian climate"... (1955/2000)
Generating text for "Indian economy"... (1992/2000)
Generating text for "Indian defense"... (1948/2000)
Generating text for "Indian climate"... (1956/2000)
Generating text for "Indian economy"... (1993/2000)
Generating text for "Indian defense"... (1949/2000)
Generating text for "Indian climate"... (1957/2000)
Generating text for "Indian defense"... (1950/2000)
Generating text for "Indian economy"... (1994/2000)
Generating text for "Indian climate"... (1958/2000)
Generating text for "Indian defense"... (1951/2000)
Generating text for "Indian economy"... (1995/2000)
Generating text for "Indian climate"... (1959/2000)
Generating text for "Indian defense"... (1952/2000)
Generating text for "Indian climate"... (1960/2000)
Generating text for "Indian economy"... (1996/2000)
Generating text for "Indian defense"... (1953/2000)
Generating text for "Indian climate"... (1961/2000)
Generating t

In [5]:
import pandas as pd
dfs = {}

for topic in TOPICS:
  df = pd.read_csv(f"{DATASET_PATH}/{topic}/data.csv")
  dfs[topic] = df

In [6]:
dfs['Indian climate']

,Prompt,Generated Text,i,MAX_LEN
0,Indian climate,Indian climate model: A comparison with climat...,0,46
1,Indian climate,"Indian climate is unique, but when the sun goe...",1,47
2,Indian climate,Indian climate is unique in that it is an extr...,2,50
3,Indian climate,"Indian climate, especially during the mid-summ...",3,51
4,Indian climate,Indian climate and land use are increasingly b...,4,55
...,...,...,...,...
1995,Indian climate,Indian climate) and the Arctic is one of the m...,1995,60
1996,Indian climate,"Indian climate,"" he said.\n\nCarmichael said t...",1996,46
1997,Indian climate,"Indian climate and human activity.""\n\n""When y...",1997,47
1998,Indian climate,Indian climate is much different from that of ...,1998,56


In [7]:
dfs['Indian economy']

,Prompt,Generated Text,i,MAX_LEN
0,Indian economy,Indian economy and it's very simple. If we hav...,0,55
1,Indian economy,Indian economy is in a state of crisis. Its mo...,1,48
2,Indian economy,"Indian economy, the nation's second largest, w...",2,49
3,Indian economy,"Indian economy, and the rest of Europe, have b...",3,60
4,Indian economy,Indian economy as we know it has been at the b...,4,48
...,...,...,...,...
1995,Indian economy,"Indian economy"" is the name given to the count...",1995,48
1996,Indian economy,Indian economy is facing a severe setback as t...,1996,53
1997,Indian economy,"Indian economy was still very small,"" he says....",1997,47
1998,Indian economy,"Indian economy, for now. A government official...",1998,59


In [8]:
dfs['Indian defense']

,Prompt,Generated Text,i,MAX_LEN
0,Indian defense,Indian defense officials recently said they ar...,0,50
1,Indian defense,Indian defense officials confirmed to Reuters ...,1,46
2,Indian defense,Indian defense minister Maj. Gen. Amir Esmail ...,2,57
3,Indian defense,Indian defense chiefs on Thursday said the U.S...,3,49
4,Indian defense,Indian defense minister Moshe Yaalon has decla...,4,59
...,...,...,...,...
1995,Indian defense,Indian defense minister Ehud Barak said Israel...,1995,49
1996,Indian defense,Indian defense minister says his army will soo...,1996,51
1997,Indian defense,Indian defense was far superior to the Japanes...,1997,47
1998,Indian defense,Indian defense minister Sergei Shoigu said on ...,1998,53


In [9]:
dfs['Indian infrastructure']

,Prompt,Generated Text,i,MAX_LEN
0,Indian infrastructure,Indian infrastructure has been damaged by floo...,0,51
1,Indian infrastructure,"Indian infrastructure, the government says.\n\...",1,52
2,Indian infrastructure,Indian infrastructure and development programm...,2,52
3,Indian infrastructure,Indian infrastructure was built on the basis o...,3,57
4,Indian infrastructure,"Indian infrastructure, the government is yet t...",4,60
...,...,...,...,...
1995,Indian infrastructure,Indian infrastructure.\n\nMr. Trump's proposal...,1995,53
1996,Indian infrastructure,Indian infrastructure will be one of the first...,1996,59
1997,Indian infrastructure,Indian infrastructure is also in the process o...,1997,53
1998,Indian infrastructure,Indian infrastructure and infrastructure proje...,1998,45
